**This notebook is copy of this one https://www.kaggle.com/steamedsheep/yolov5-is-all-you-need I just follow the advices from discussion and changed the input size from 3000 to 10000**

# Install necesessary libraries

In [ ]:
# norfair dependencies
%cd /kaggle/input/norfair031py3/
!pip install commonmark-0.9.1-py2.py3-none-any.whl -f ./ --no-index
!pip install rich-9.13.0-py3-none-any.whl

!mkdir /kaggle/working/tmp
!cp -r /kaggle/input/norfair031py3/filterpy-1.4.5/filterpy-1.4.5/ /kaggle/working/tmp/
%cd /kaggle/working/tmp/filterpy-1.4.5/
!pip install .
!rm -rf /kaggle/working/tmp

# norfair
%cd /kaggle/input/norfair031py3/
!pip install norfair-0.3.1-py3-none-any.whl -f ./ --no-index


In [ ]:
%cd /kaggle/working

import cv2
import sys
import torch
import ast
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.notebook import tqdm
tqdm.pandas()
from norfair import Detection, Tracker

from PIL import Image
from IPython.display import display

sys.path.append('../input/tensorflow-great-barrier-reef')
sys.path.append('../input/weightedboxesfusion/')
sys.path.append('../input/boxes-mask')
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')

import timm
from fastai.vision.all import *
from ensemble_boxes.ensemble_boxes_wbf import weighted_boxes_fusion
from boxes_mask import box_voting

!mkdir -p /root/.config/Ultralytics
!cp /kaggle/input/yolov5-font/Arial.ttf /root/.config/Ultralytics/

import greatbarrierreef
env = greatbarrierreef.make_env()# initialize the environment
iter_test = env.iter_test()      # an iterator which loops over the test set and sample submission

# Load Model

In [ ]:
def _load_model(path, conf, iou):
    model = torch.hub.load('../input/yolov5-lib-ds', 
                          'custom', 
                          path = path,
                          source='local',
                          force_reload=True)  # local repo
    model.conf = conf
    model.iou = iou
    model.classes = None   # (optional list) filter by class, i.e. = [0, 15, 16] for persons, cats and dogs
    model.multi_label = False  # NMS multiple labels per box
    model.max_det = 1000  # maximum number of detections per image
    return model


path1 = '../input/yolov5s6-1920-b8-10pct-adam-35ep-video/f0.pt'
model1 = _load_model(path1, 0.1, 0.4)
path2 = '../input/yolov5s6-1920-b8-10pct-adam-35ep-video/f1.pt'
model2 = _load_model(path2, 0.1, 0.4)
path3 = '../input/yolov5s6-1920-b8-10pct-adam-35ep-video/f2.pt'
model3 = _load_model(path3, 0.1, 0.4)

# Inference utils

In [ ]:
def is_cots(learner, img, bbox, bbox_conf, conf_ratio=0.5, visualize=False):   
    cropped_img = img[bbox[1] : bbox[3], bbox[0] : bbox[2]]
#     cropped_img = Img.fromarray(cropped_img).resize((240, 200))
#     cropped_img = np.asarray(cropped_img)
    
    with learner.no_bar():
        preds = learner.predict(cropped_img)
    pred_conf = preds[2][0]
    
    conf = bbox_conf + (pred_conf-0.5) * conf_ratio
    conf = max(conf, 0)
    conf = min(conf, 1)
    
    if visualize:
        display(Img.fromarray(cropped_img))
        print(bbox_conf, bbox)
        print(preds)
        print(conf)
        print('+'*80)
    
    return conf

# Tracking utils

In [ ]:
##############################################################
#                      Tracking helpers                      #
##############################################################

import numpy as np
from norfair import Detection, Tracker

# Helper to convert bbox in format [x_min, y_min, x_max, y_max, score] to norfair.Detection class
def to_norfair(detects, frame_id):
    result = []
    for x_min, y_min, x_max, y_max, score in detects:
        xc, yc = (x_min + x_max) / 2, (y_min + y_max) / 2
        w, h = x_max - x_min, y_max - y_min
        result.append(Detection(points=np.array([xc, yc]), scores=np.array([score]), data=np.array([w, h, frame_id])))
        
    return result

# Euclidean distance function to match detections on this frame with tracked_objects from previous frames
def euclidean_distance(detection, tracked_object):
    return np.linalg.norm(detection.points - tracked_object.estimate)


#######################################################
#                      Tracking                       #
#######################################################

# Tracker will update tracks based on detections from current frame
# Matching based on euclidean distance between bbox centers of detections 
# from current frame and tracked_objects based on previous frames
# You can check it's parameters in norfair docs
# https://github.com/tryolabs/norfair/blob/master/docs/README.md
tracker = Tracker(
    distance_function=euclidean_distance, 
    distance_threshold=30,
    hit_inertia_min=3,
    hit_inertia_max=6,
    initialization_delay=1,
)


# Prepare validation data

In [ ]:
# def get_bbox(annots):
#     bboxes = [list(annot.values()) for annot in annots]
#     return bboxes

# def get_path(row):
#     row['image_path'] = f'/kaggle/input/tensorflow-great-barrier-reef/train_images/video_{row.video_id}/{row.video_frame}.jpg'
#     return row

# df = pd.read_csv('/kaggle/input/tensorflow-great-barrier-reef/train.csv')
# df['annotations'] = df['annotations'].apply(lambda x: ast.literal_eval(x))
# df['num_bbox'] = df['annotations'].apply(lambda x: len(x))
# df['bboxes'] = df.annotations.progress_apply(get_bbox)
# df = df.apply(get_path, axis=1)

In [ ]:
# image_paths = df[df.num_bbox>4].sample(100)
# image_paths.head()

# Ensembling with WBF + Tracking

In [ ]:
IMG_SIZE = 1280
iou_thr = 0.5
min_box_size = 0
conf_thr = 0.6
conf_ratio = 0.7

COCO_CLASSES = (
                  "starfish",
                )

submission_dict = {
    'id': [],
    'prediction_string': [],
}

# WBF
def run_wbf(bboxes, scores, orig_image_size=IMG_SIZE, iou_thr=iou_thr, skip_box_thr=0.0, weights=None):
    # normalize bboxes
    for i in range(len(bboxes)):
        for j in range(len(bboxes[i])):
            for k in range(4):
                bboxes[i][j][k] /= (orig_image_size-1)
    labels = [np.ones(len(score)) for score in scores]
    # fuse bboxes
    bboxes, scores, labels = weighted_boxes_fusion(bboxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    # return bboxes to the original size
    bboxes = [bbox*(orig_image_size-1) for bbox in bboxes]
    return bboxes, scores

# Box voting
def run_bbox_voting(bboxes, scores, all_bboxes, thresh=iou_thr, scoring_method='IOU_WAVG', beta=1.0, mask=None):
    top_bboxes =  np.empty((0,5), float)
    for bbox, score in zip(bboxes, scores):
        top_bbox = np.array([[bbox[0], bbox[1], bbox[2], bbox[3], score]])
        top_bboxes = np.append(top_bboxes, top_bbox, axis=0)
    bboxes = box_voting(top_bboxes, all_bboxes, thresh, scoring_method, beta, mask)
    bboxes, scores = bboxes[:, :4], bboxes[:, 4]
    return bboxes, scores

def draw_predictions(img, bboxes, scores, classes_dict):
    for i in range(len(bboxes)):
        box = bboxes[i]
        cls_id = 0
        score = scores[i]
        x0 = int(box[0])
        y0 = int(box[1])
        x1 = int(box[2])
        y1 = int(box[3])

        cv2.rectangle(img, (x0, y0), (x1, y1), (0, 255, 0), 2)
        cv2.putText(img, '{}:{:.1f}%'.format(classes_dict[cls_id], score * 100), (x0, y0 - 3), cv2.FONT_HERSHEY_PLAIN, 0.8, (0,255,0), thickness = 1)
    return img

# Set parameters for fastai binary image classificator 
learner_resnet18 = load_learner(f'../input/fastai-models/resnet18.pkl')
# Save frame_id into detection to know which tracks have no detections on current frame
frame_id = 0

global bboxes, scores, labels
# for idx, row in tqdm(image_paths.iterrows()):
#     img = cv2.imread(row.image_path)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
for img, pred_df in tqdm(iter_test):
    predictions, detects = [], []
    bboxes, scores, labels = [], [], []
    all_bboxes = np.empty((0,5), float)
    
    r1 = model1(img, size=2700, augment=True)
    r2 = model2(img, size=2700, augment=True)
    r3 = model3(img, size=2700, augment=True)
    
    def get_scores_labels_boxes(model_output):
        global bboxes, scores, labels, all_bboxes
        output = model_output.pandas().xyxy[0]
        if output.shape[0] != 0:
            bbox, score, label = [], [], []
            for idx, row in output.iterrows():
                # Skip too small boxes
                len_x, len_y = row['xmax'] - row['xmin'], row['ymax'] - row['ymin']
                if len_x < min_box_size or len_y < min_box_size:
                    continue
                
                all_bboxes = np.append(all_bboxes, np.array([row[['xmin', 'ymin', 'xmax', 'ymax', 'confidence']].values]), axis=0)
                bbox.append(row[['xmin', 'ymin', 'xmax', 'ymax']].values)
                score.append(row['confidence'])
                label.append(row['class'])
            
            if bbox:
                bboxes.append(bbox)
                scores.append(score)
                labels.append(label)   
                                   
    get_scores_labels_boxes(r1)
    get_scores_labels_boxes(r2)
    get_scores_labels_boxes(r3)

    if len(bboxes) > 0:
        bboxes, scores = run_wbf(bboxes, scores)
        bboxes, scores = run_bbox_voting(bboxes, scores, all_bboxes)
        for bbox, score in zip(bboxes, scores):
            xmin, ymin, xmax, ymax = int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3])
            bbox_conf = is_cots(learner_resnet18, img, [xmin, ymin, xmax, ymax], score, conf_ratio, visualize=False) 
            
            if bbox_conf > conf_thr:
                width, height = xmax-xmin, ymax-ymin
                detects.append([xmin, ymin, xmax, ymax, score])
                predictions.append('{:.2f} {} {} {} {}'.format(score, xmin, ymin, width, height))

    #######################################################
    #                      Tracking                       #
    #######################################################
    
    # Update tracks using detects from current frame
    tracked_objects = tracker.update(detections=to_norfair(detects, frame_id))
    for tobj in tracked_objects:
        bbox_width, bbox_height, last_detected_frame_id = tobj.last_detection.data
        # Skip objects that were detected on current frame
        if last_detected_frame_id == frame_id:  
            continue
        # Add objects that have no detections on current frame to predictions
        xc, yc = tobj.estimate[0]
        x_min, y_min = int(round(xc - bbox_width / 2)), int(round(yc - bbox_height / 2))
        score = tobj.last_detection.scores[0]
        # Skip too small boxes
        if bbox_width < min_box_size or bbox_height < min_box_size:
            continue
        
        predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
    #######################################################
    
    prediction_str = ' '.join(predictions)
    pred_df['annotations'] = prediction_str
    env.predict(pred_df)

#     print('Prediction:', prediction_str)
    frame_id += 1

    # Draw predictions
#     out_image = draw_predictions(img, bboxes, scores, COCO_CLASSES)

    # Since we load image using OpenCV we have to convert it 
#     out_image = cv2.cvtColor(out_image, cv2.COLOR_BGR2RGB)
#     display(Image.fromarray(out_image))

In [ ]:
sub_df = pd.read_csv('submission.csv')
sub_df.head(3)